# Toolkit Showcase: Prepare and run a protein-ligand simulation

<br />
<details>
    <summary><small>▼ Click here for dependency installation instructions</small></summary>

    # The simplest way to install dependencies is to install the examples package:
    
    conda install -c conda-forge openff-toolkit-examples
    
    # This example will need access to a GROMACS install in addition to the above dependencies. Your existing GROMACS installed can be used, or you can install it from Bioconda:
    
    conda install -c conda-forge -c bioconda gromacs
    
    # You can also install all the depencies using the provided environment.yaml:
    
    conda env update --file ../environment.yaml 
    
    # You may also need to restart this notebook's kernel after you make these changes (Kernel -> Restart)
</details>


Welcome to the Open Force Field Toolkit! This introductory example will demonstrate how to prepare a system for Molecular Dynamics that combines solvent, a ligand using the Parsley open force field, and a protein using a standard Amber force field. We'll use a number of tools from the Python molecular science ecosystem to take a pre-docked benchmark example from scratch all the way through to simulation and visualization, all without leaving the notebook. Have fun!

In [1]:
%%html
<!--" Stylesheet for improved readability. This example has a lot of reading, so we want good typography. "-->
<style>
p, ul, ol, h1, h2, h3, h4, h5, h6 {
    max-width: 50rem;    
}
</style>

In [2]:
# Imports from the Python standard library
import sys
from pathlib import Path
from tempfile import NamedTemporaryFile

# Imports from dependencies
from simtk import openmm, unit
import parmed as pmd
import numpy as np
import mdtraj as mdt
import nglview
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

# Imports from the toolkit
import openff.toolkit
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.toolkit.topology import Molecule, Topology


_(The OpenEye loading warning is expected -- The toolkit is informing us that OETK is unavailable, but it will safely fall back to using RDKit and AmberTools for the same functionality)_

## Introducing the main cast

Merck [provides data](https://github.com/MCompChem/fep-benchmark) to benchmark Free Energy Perturbation (FEP) procedures. We'll use structures from this dataset for this showcase:

https://github.com/MCompChem/fep-benchmark

This example is pre-packaged with one protein-ligand complex from the above repository, however you should be able to download other complexes and run them as well using this workflow. CHEMBL1078774, our ligand of choice, is an inhibitor of the mitotic functions of kinesin-5, a motor protein involved in cell division. 

The ligand and protein structures are already prepared for simulation:

- Their co-ordinates are super-imposable
- Hydrogens have been added to protein and crystallographic waters
- The protein's termini are capped with N-methyl and acetyl groups to prevent unphysical charges
- Atoms missing from the crystal structure have been replaced

<br />
<details>
    <summary><small>▼ Click here for the shell commands we used to download the protein-ligand complex</small></summary>

    # Clone the repository
    git clone https://github.com/MCompChem/fep-benchmark.git
    # Take the first ligand from the eg5 benchmark
    head -n119 fep-benchmark/eg5/ligands.sdf > chembl_1078774.sdf
    # Take the prepared protein structure
    cp fep-benchmark/eg5/3l9h_prepared.pdb .
    
</details>

In [3]:
receptor_path = "3l9h_prepared.pdb"
ligand_path = "chembl_1078774.sdf"

We can visualize each structure using the [NGLView](https://github.com/nglviewer/nglview) widget. These visualizations are interactive; rotate by dragging the left mouse button, pan with the right mouse button, and zoom with the scroll wheel. You can also mouse over an atom to see its details, and click an atom to center the view on it. When you mouse over the widget, a full screen button will appear in its top right corner. 

In [4]:
view = nglview.show_file(ligand_path)
view

NGLWidget()

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ Try replacing <code>ligand_path</code> with <code>receptor_path</code> to visualize the protein!
</div>


# The plan:

| Action | Software|
|--|--|
| Assemble the force field | OpenFF Toolkit, OpenMM Force Fields, OpenMM
| Model and solvate the complex | OpenMM
| Parameterize the complex | OpenMM (OpenFF Toolkit is under the hood)
| Visualize the complex | NGLView
| Simulate the complex | OpenMM
| Visualize the simulation | NGLView


## Assemble the force field (OpenFF Toolkit, OpenMM Force Fields, OpenMM)

The first step in this workflow, as with any simulation, is to choose the force field. The OpenMM Force Fields package ([`openmmforcefields`](https://github.com/openmm/openmmforcefields)) allows us to combine force fields for different components so that we can model the system in one go.

The star of the show here is the Parsley 1.3.0 force field. [Parsley](https://openforcefield.org/force-fields/force-fields/#parsley) is the first generation force field produced by the Open Force Field Initiative. Rather than using atom types like traditional biomolecular force fields, Parsley assigns parameters to a molecule with [fancy subgraph matching](https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html). Note that to parameterize a molecule you need more than just the co-ordinates of its atoms; you also need their bonds, bond orders, and formal charges. As a result, `.sdf` files are used in this example; other file types are possible, but they must include this information. Parsley is distributed as an `.offxml` file according to the [SMIRNOFF specification](https://open-forcefield-toolkit.readthedocs.io/en/latest/smirnoff.html). This file describes all the choices the toolkit has to make to parameterize a molecule.

The Parsley force field is designed for small molecule parameterization. Parameters for proteins and other polymers are coming, but for now we'll use Amber 99sb as our protein force field. The protein is already prepared with Amber-compatible residue names, so this is simple. Finally, we'll use the TIP3p water model, as both Parsley and Amber are parameterized for it.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that OpenMM and the Open Force Field Toolkit both have classes called `Topology` and `ForceField` that serve similar functions. Don't get them confused!
</div>


In [5]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)
# Molecule stores both the co-ordinates of the atoms and their bond graph
ligand_positions = ligand.conformers[0]
ligand_topology = ligand.to_topology()

# Load protein and water force field parameters
omm_forcefield = openmm.app.ForceField(
    "amber99sb.xml", "tip3p.xml"
)
# Teach OpenMM about the ligand molecule and the Parsley force field
smirnoff = SMIRNOFFTemplateGenerator(forcefield="openff-1.3.0.offxml", molecules=[ligand])
omm_forcefield.registerTemplateGenerator(smirnoff.generator)

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ This is the only block in the example that uses the Open Force Field Toolkit directly!
</div>

## Model and solvate the complex (OpenMM)

Conceptually, this step involves putting together the positions of all of the components of the system. In practice, this is also where the Parsley parameters for the ligand are actually generated. This involves computing partial charges for its atoms, which takes time.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Note that 4 Å is not enough padding for regular simulations; we're just using it for a fast example.
</div>

In [6]:
# Load the kinesin-5 receptor structure into Modeller
pdb = openmm.app.PDBFile(receptor_path)
modeller = openmm.app.Modeller(pdb.topology, pdb.positions)

# Add the ligand
# Under the hood, this line uses the OpenFF Toolkit to generate new parameters for the ligand
modeller.add(ligand_topology.to_openmm(), ligand_positions)

# solvate it in 0.15 M NaCl solution
modeller.addSolvent(
    omm_forcefield,
    model="tip3p",
    padding=4.0 * unit.angstrom,
    ionicStrength=0.15 * unit.molar,
)


_(This'll take a minute or two)_

In this workflow, the Toolkit is just responsible for combining a force field with a molecular topology, and much of this work happens behind the scenes through the OpenMM ForceField object. The toolkit is designed to check the input and give the user useful feedback if there seem to be errors or problems in the provided files. 

    
### Charge generation

In this step, the toolkit also computes partial charges without user intervention. Computing charges is a process that can be confusing and error-prone, so we try to specify and simplify it as much as possible. Charges are computed efficiently with [OpenEye](https://www.eyesopen.com/) if is available; if it is not, the free toolkits [RDKit](https://www.rdkit.org/) and [AmberTools](https://ambermd.org/AmberTools.php) are used instead.

So far, released OpenFF force fields only use AM1-BCC partial charge generation. Other semi-empirical methods and charge corrections are available in the toolkit, and they'll be used in future force fields. In particular, so-called "graph based" charges are coming soon! Graph-based charges use only the bond graph to compute charges, rather than requiring 3D coordinates. This promises to make the produced partial charges independent of the conformation (a feature we currently provide at [some cost](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-dont-seem-to-depend-on-the-molecules-conformation-is-this-a-bug)), and may even be a lot faster to boot.

The toolkit also supports [library charges](https://open-forcefield-toolkit.readthedocs.io/en/latest/api/generated/openff.toolkit.typing.engines.smirnoff.parameters.LibraryChargeHandler.html#openff.toolkit.typing.engines.smirnoff.parameters.LibraryChargeHandler) that can be assigned via a template.
    

## Parameterize the complex (OpenMM, OpenFF Toolkit under the hood)

This step assigns parameters to the solvent and protein, and combines them with the ligand parameters into an OpenMM `System` object, which encodes all the forces and interactions in the molecular system.

In [7]:
# Construct the OpenMM System, which stores the forces for the system
system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    nonbondedCutoff=9 * unit.angstrom,
    constraints=openmm.app.HBonds,
    rigidWater=True,
)

# Retrieve the OpenMM Topology, which stores the atoms and connectivity
topology = modeller.getTopology()

# Get the initial positions
# The box is about 75 angstroms per side, so add (30, 30, 30) to center the protein
positions = modeller.getPositions() + np.array([30, 30, 30]) * unit.angstrom

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ⚠️ Note that the <code>nonbondedMethod</code>, <code>nonbondedCutoff</code>, <code>constraints</code>
     and <code>rigidWater</code> arguments to the <code>createSystem</code> method are all parameters of
    the force field! In principle we could collect them from the <code>openff-1.3.0.offxml</code> file, but
    since this would add a lot of complexity to this example, in this case we don't.
</div>

## Visualize the complex (NGLView)

To visualize inside the notebook, we'll use NGLView. NGLView supports a wide variety of [molecular visualization methods](https://nglviewer.org/ngl/api/manual/molecular-representations.html), as well as a VMD-like [atom selection language](https://nglviewer.org/ngl/api/manual/selection-language.html). This can be used to visualize complex systems like this one.

The widget consists of a minimally documented [Python library frontend](https://nglviewer.org/nglview/latest/api.html) and an extensively documented [JavaScript backend](https://nglviewer.org/ngl/api/manual/index.html). You'll need to refer to the documentation for both to do anything sophisticated, as the Python code delegates most of its options and functionality to the JS code.

In [8]:
# Create an MDTraj Trajectory (with only one step), which NGLView knows how to read
mdt_traj = mdt.Trajectory(positions, mdt.Topology.from_openmm(topology))

# Create the widget. By default, proteins are shown as a cartoon and unrecognised ligands with a ball-and-stick model
view = nglview.show_mdtraj(mdt_traj)
    
# Add a licorice/stick representation for everything except the protein
view.add_licorice(selection="(not protein)")
# Make the ions clearer by drawing their vdW surfaces
view.add_surface(selection=":.NA or :.CL")

# Render the widget
view

NGLWidget()

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ Have a play with this visualization! Try clearing the default representations with <code>view.clear()</code> and configuring your own cartoon <em>(Hint: <a href=https://nglviewer.org/nglview/latest/api.html#nglview.NGLWidget>Check</a> the <a href=https://nglviewer.org/ngl/api/manual/molecular-representations.html>docs</a>)</em>. See if you can display the ligand in a way you like, without also displaying the protein's terminal caps. When you're happy with what you've made, save the image with <code>view.download_image()</code>
</div>

## Simulate the complex (OpenMM)

All that remains is to tell OpenMM the details about how we want to integrate and record data for the simulation, and then to put everything together and run it! 

### Configure and run the simulation

Here, we'll use a Langevin thermostat at 300 Kelvin and a 2 fs time step. We'll write the structure to disk every 10 steps.

In [9]:
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * unit.kelvin, 1 / unit.picosecond, 0.002 * unit.picoseconds
)

# Combine the topology, system, integrator and initial positions into a simulation
simulation = openmm.app.Simulation(
    topology, system, integrator
)
simulation.context.setPositions(positions)

# Add a reporter to record the structure every 10 steps
nc_reporter = pmd.openmm.NetCDFReporter("trajectory.nc", 10)
simulation.reporters.append(nc_reporter)

### Minimize the combined system

This will reduce any forces that are too large to integrate, such as from clashes or from disagreements between the crystal structure and force field.


In [10]:
simulation.minimizeEnergy()
minimized_state = simulation.context.getState(
    getPositions=True,
    getEnergy=True,
    getForces=True
)

print(
    "Minimised to", 
    minimized_state.getPotentialEnergy(), 
    "with maximum force", 
    max([np.sqrt(v.x*v.x + v.y*v.y + v.z*v.z) for v in minimized_state.getForces()]),
    minimized_state.getForces().unit.get_symbol()
)

minimized_coords = minimized_state.getPositions()

Minimised to -650074.5332745998 kJ/mol with maximum force 2518.6278680277846 kJ/(nm mol)


_(This'll take a minute or two)_

### Run a short simulation

If this were anything more than a demonstration of the Toolkit, this example would need to include additional steps like equilibration. 

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [11]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)
simulation.step(1000)

_(This'll take a minute or two, we're writing to disk very frequently)_

### Open Source Force Fields

A primary goal of the Open Force Field Initiative is to make development and use of force fields as open as possible - it's in our name! We believe that open source development practices have a lot to offer the scientific community, whether that science is academic, commercial, or hobbyist.

#### The SMIRNOFF specification

The SMIRNOFF specification describes a simple format for describing molecular force fields. We provide and maintain this spec in the hopes that it will allow scientists everywhere to contribute to force field development in a unified way, without taking them away from their favourite simulation package.

SMIRNOFF is not just a spec; we're also committed to a reference implementation — that being the OpenFF Toolkit. The Toolkit endeavors to support all the functional forms in both the SMIRNOFF spec and the [`openff-forcefields`](https://github.com/openforcefield/openff-forcefields/) package.

#### Reproducibility

OpenFF force fields are completely specified by the name of the distributed `.offxml` file. We use codenames, version numbers, and tags to accomplish this. This means that as long as a user, designer, or reviewer sees the name of the force field being used, they know exactly what is going in to that simulation. We include parameters that are often treated as implementation details in this specification, so details like the cut-off distance, constraints, modifications to the Lennard-Jones function, and partial charge generation methods are all defined by the name of the force field. 

As much as possible, we want energy and force to be a deterministic output of combining a molecule and a force field. If an author provides the name of the force field in their methods section, it should be reproducible. The other side of this coin is that we never want to hide the force field from the user. In all our workflows, the name of the force field must be explicitly provided in the code. This improves reproducibility of the code and helps the user take responsibility for their results. 

#### "Plugin" support for new force fields

The OpenFF Toolkit supports distributing force field files (.offxml) through Conda data packages. Anyone can publish a package on Conda Forge that extends the list of directories the toolkit searches for force fields, allowing anyone to produce force fields without requiring their own tooling, in a format that is designed to be converted to a multitude of simulation packages. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#how-can-i-distribute-my-own-force-fields-in-smirnoff-format) for more details.


## Visualize the simulation (nglview)

NGLView can display trajectories, as well as single structures. Mouse over the widget to see the animation controls.

In [12]:
nc_reporter.finalize()
mdt_traj = mdt.load("trajectory.nc", top=mdt.Topology.from_openmm(topology))

view = nglview.show_mdtraj(mdt_traj)
view

NGLWidget(max_frame=99)

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ MDTraj is a great library for analysis as well as visualisation. Check out the <a href=https://mdtraj.org/1.9.4/api/generated/mdtraj.Trajectory.html>docs</a> for the <code>Trajectory</code> object you just created, as well as their <a href=https://mdtraj.org/1.9.4/analysis.html>analysis functions</a>, and see if you can compute something interesting. Its real superpower is that it provides the coordinates of the trajectory as a <a href=https://numpy.org/doc/stable/reference/generated/numpy.array.html>NumPy array</a>, so if you're really keen try computing something directly from <code>mdt_traj.xyz</code>
</div>

## Conclusions

* Toolkit parameterization of an unknown ligand slides right in alongside specifying the protein and solvent force fields
* Open source tools installed via Conda did everything, from basic system prep to simulation and visualization
* Using OpenMM, we never had to leave Python or touch the disk to set up the simulation
* With ParmEd, using GROMACS, Amber or CHARMM is simple! See the appendix for details.


## What's next?

We have more examples for you to explore the toolkit!

 - [Force Field Modification](https://github.com/openforcefield/openff-toolkit/blob/master/examples/forcefield_modification/): Use the Toolkit API to manipulate SMIRNOFF parameters in a `ForceField` object
 - [Conformer energies](https://github.com/openforcefield/openff-toolkit/blob/master/examples/conformer_energies/): Compute vacuum energies of different conformers of a small molecule with the Parsley force field
 - [QCArchive interface](https://github.com/openforcefield/openff-toolkit/blob/master/examples/QCArchive_interface/): Create OpenFF `Molecule` objects from the [QCArchive](https://qcarchive.molssi.org/)

# Appendix: What about GROMACS?

OpenMM makes it easy to run molecular simulations without leaving Python. The OpenFF toolkit currently exports directly to OpenMM, but no part of the parametrization process is exclusively supported by OpenMM. With ParmEd and other tools, the same systems can be run in other engines. Here we show how to use ParmEd to prepare and run the same workflow in GROMACS.

We have [another example](https://github.com/openforcefield/openff-toolkit/blob/master/examples/using_smirnoff_in_amber_or_gromacs/convert_to_amber_gromacs.ipynb) that uses ParmEd to produce Amber input files too, and also to validate single point energies.


[ParmEd](https://github.com/ParmEd/ParmEd) is able to read, write, combine, and edit topologies and structures from several different simulation programs, including CHARMM, OpenMM, Amber, and GROMACS. Here, we'll use it to convert our OpenMM `System` to a format we can visualize, and later we'll use it to convert our prepared system to another simulation package.

ParmEd doesn't support constraints from OpenMM because different simulation packages have conflicting ways of specifying them. So we need to create a new OpenMM `System` object here, and read it into a ParmEd [`Structure`](https://parmed.github.io/ParmEd/html/structure.html) (see also [API docs](https://parmed.github.io/ParmEd/html/structobj/parmed.structure.Structure.html))


In [13]:
# ParmEd's GROMACS exporter can't handle constraints from openmm, so we need a variant for export without them
export_system = omm_forcefield.createSystem(
    modeller.topology, 
    nonbondedMethod=openmm.app.PME,
    constraints=None,
    rigidWater=False,
)

# Combine the topology, system and positions into a ParmEd Structure
pmd_complex_struct = pmd.openmm.load_topology(
    topology, export_system, positions
)

# Export GROMACS files.
pmd_complex_struct.save("system.top", overwrite=True)
pmd_complex_struct.save("system.gro", overwrite=True)

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ⚠️ Like before, the <code>nonbondedMethod</code>, <code>constraints</code>
    and <code>rigidWater</code> arguments to the <code>createSystem</code> method, 
    as well as the implicit cutoff distance, are all specified in the force field. 
    So, we need to make sure our MDP file reflects the correct values.
</div>

Energy minimize the system. Unfortunately the net charge is not zero; this is a [known bug](https://github.com/openforcefield/openff-toolkit/issues/456) in the toolkit.

In [14]:
# Configure GROMACS not to clutter up our directory with backups
%env GMX_MAXBACKUP=-1

# Energy minimise the coordinates
! gmx grompp -f minim.mdp -c system.gro -p system.top -o em.tpr -maxwarn 1
! gmx mdrun -v -deffnm em


env: GMX_MAXBACKUP=-1
                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meul

Step=   31, Dmax= 1.7e-02 nm, Epot= -5.12999e+05 Fmax= 1.50661e+04, atom= 2434
Step=   32, Dmax= 2.1e-02 nm, Epot= -5.14053e+05 Fmax= 1.60277e+04, atom= 2434
Step=   33, Dmax= 2.5e-02 nm, Epot= -5.14191e+05 Fmax= 2.18954e+04, atom= 2434
Step=   34, Dmax= 3.0e-02 nm, Epot= -5.15061e+05 Fmax= 2.28452e+04, atom= 2434
Step=   36, Dmax= 1.8e-02 nm, Epot= -5.18425e+05 Fmax= 4.37113e+03, atom= 2434
Step=   38, Dmax= 1.1e-02 nm, Epot= -5.19661e+05 Fmax= 1.13764e+04, atom= 2434
Step=   39, Dmax= 1.3e-02 nm, Epot= -5.20772e+05 Fmax= 8.38195e+03, atom= 2434
Step=   40, Dmax= 1.5e-02 nm, Epot= -5.21288e+05 Fmax= 1.47065e+04, atom= 2434
Step=   41, Dmax= 1.8e-02 nm, Epot= -5.22298e+05 Fmax= 1.36552e+04, atom= 2434
Step=   42, Dmax= 2.2e-02 nm, Epot= -5.22314e+05 Fmax= 1.97400e+04, atom= 2434
Step=   43, Dmax= 2.7e-02 nm, Epot= -5.22963e+05 Fmax= 2.08981e+04, atom= 2434
Step=   45, Dmax= 1.6e-02 nm, Epot= -5.25474e+05 Fmax= 3.22289e+03, atom= 3720
Step=   46, Dmax= 1.9e-02 nm, Epot= -5.25505e+05 Fma

Step=  162, Dmax= 9.2e-03 nm, Epot= -5.69442e+05 Fmax= 1.12260e+04, atom= 1019
Step=  163, Dmax= 1.1e-02 nm, Epot= -5.69695e+05 Fmax= 5.62664e+03, atom= 1019
Step=  164, Dmax= 1.3e-02 nm, Epot= -5.69719e+05 Fmax= 1.45316e+04, atom= 1019
Step=  165, Dmax= 1.6e-02 nm, Epot= -5.70014e+05 Fmax= 9.71201e+03, atom= 1019
Step=  167, Dmax= 9.5e-03 nm, Epot= -5.70229e+05 Fmax= 4.80006e+03, atom= 1019
Step=  168, Dmax= 1.1e-02 nm, Epot= -5.70311e+05 Fmax= 1.26974e+04, atom= 1019
Step=  169, Dmax= 1.4e-02 nm, Epot= -5.70566e+05 Fmax= 8.21653e+03, atom= 1019
Step=  171, Dmax= 8.2e-03 nm, Epot= -5.70750e+05 Fmax= 4.36275e+03, atom= 1019
Step=  172, Dmax= 9.8e-03 nm, Epot= -5.70874e+05 Fmax= 1.06745e+04, atom= 1019
Step=  173, Dmax= 1.2e-02 nm, Epot= -5.71081e+05 Fmax= 7.43776e+03, atom= 1019
Step=  174, Dmax= 1.4e-02 nm, Epot= -5.71089e+05 Fmax= 1.42330e+04, atom= 1019
Step=  175, Dmax= 1.7e-02 nm, Epot= -5.71310e+05 Fmax= 1.18150e+04, atom= 1019
Step=  177, Dmax= 1.0e-02 nm, Epot= -5.71580e+05 Fma

Step=  293, Dmax= 1.2e-02 nm, Epot= -5.85457e+05 Fmax= 1.85603e+03, atom= 2031
Step=  294, Dmax= 1.4e-02 nm, Epot= -5.85578e+05 Fmax= 1.96265e+04, atom= 2031
Step=  295, Dmax= 1.7e-02 nm, Epot= -5.85925e+05 Fmax= 6.52203e+03, atom= 2031
Step=  297, Dmax= 1.0e-02 nm, Epot= -5.85983e+05 Fmax= 9.01282e+03, atom= 2031
Step=  298, Dmax= 1.2e-02 nm, Epot= -5.86051e+05 Fmax= 9.77123e+03, atom= 2031
Step=  299, Dmax= 1.5e-02 nm, Epot= -5.86085e+05 Fmax= 1.26102e+04, atom= 2031
Step=  300, Dmax= 1.7e-02 nm, Epot= -5.86128e+05 Fmax= 1.43790e+04, atom= 2031
Step=  302, Dmax= 1.0e-02 nm, Epot= -5.86334e+05 Fmax= 1.74769e+03, atom= 2031
Step=  303, Dmax= 1.3e-02 nm, Epot= -5.86502e+05 Fmax= 1.78680e+04, atom= 2031
Step=  304, Dmax= 1.5e-02 nm, Epot= -5.86773e+05 Fmax= 5.42093e+03, atom= 2031
Step=  306, Dmax= 9.1e-03 nm, Epot= -5.86834e+05 Fmax= 8.61974e+03, atom= 2031
Step=  307, Dmax= 1.1e-02 nm, Epot= -5.86911e+05 Fmax= 8.11280e+03, atom= 2031
Step=  308, Dmax= 1.3e-02 nm, Epot= -5.86943e+05 Fma

Step=  424, Dmax= 1.5e-02 nm, Epot= -5.95056e+05 Fmax= 1.27888e+04, atom= 2031
Step=  425, Dmax= 1.8e-02 nm, Epot= -5.95081e+05 Fmax= 1.48968e+04, atom= 2031
Step=  427, Dmax= 1.1e-02 nm, Epot= -5.95221e+05 Fmax= 1.73532e+03, atom= 2031
Step=  428, Dmax= 1.3e-02 nm, Epot= -5.95338e+05 Fmax= 1.83792e+04, atom= 2031
Step=  429, Dmax= 1.6e-02 nm, Epot= -5.95525e+05 Fmax= 5.69108e+03, atom= 2031
Step=  431, Dmax= 9.3e-03 nm, Epot= -5.95565e+05 Fmax= 8.69669e+03, atom= 2031
Step=  432, Dmax= 1.1e-02 nm, Epot= -5.95615e+05 Fmax= 8.57427e+03, atom= 2031
Step=  433, Dmax= 1.3e-02 nm, Epot= -5.95637e+05 Fmax= 1.21253e+04, atom= 2031
Step=  434, Dmax= 1.6e-02 nm, Epot= -5.95679e+05 Fmax= 1.27181e+04, atom= 2031
Step=  436, Dmax= 9.7e-03 nm, Epot= -5.95787e+05 Fmax= 2.17674e+03, atom= 2031
Step=  437, Dmax= 1.2e-02 nm, Epot= -5.95865e+05 Fmax= 1.56880e+04, atom= 2031
Step=  438, Dmax= 1.4e-02 nm, Epot= -5.96006e+05 Fmax= 5.78686e+03, atom= 2031
Step=  440, Dmax= 8.4e-03 nm, Epot= -5.96051e+05 Fma

Step=  555, Dmax= 8.0e-03 nm, Epot= -6.01898e+05 Fmax= 6.82295e+03, atom= 2031
Step=  556, Dmax= 9.6e-03 nm, Epot= -6.01931e+05 Fmax= 7.97528e+03, atom= 2031
Step=  557, Dmax= 1.2e-02 nm, Epot= -6.01955e+05 Fmax= 9.77583e+03, atom= 2031
Step=  558, Dmax= 1.4e-02 nm, Epot= -6.01977e+05 Fmax= 1.15259e+04, atom= 2031
Step=  559, Dmax= 1.7e-02 nm, Epot= -6.01986e+05 Fmax= 1.40003e+04, atom= 2031
Step=  561, Dmax= 9.9e-03 nm, Epot= -6.02105e+05 Fmax= 1.33845e+03, atom= 2031
Step=  562, Dmax= 1.2e-02 nm, Epot= -6.02213e+05 Fmax= 1.71868e+04, atom= 2031
Step=  563, Dmax= 1.4e-02 nm, Epot= -6.02371e+05 Fmax= 5.01282e+03, atom= 2031
Step=  565, Dmax= 8.6e-03 nm, Epot= -6.02398e+05 Fmax= 8.24662e+03, atom= 2031
Step=  566, Dmax= 1.0e-02 nm, Epot= -6.02436e+05 Fmax= 7.67867e+03, atom= 2031
Step=  567, Dmax= 1.2e-02 nm, Epot= -6.02447e+05 Fmax= 1.14029e+04, atom= 2031
Step=  568, Dmax= 1.5e-02 nm, Epot= -6.02482e+05 Fmax= 1.15029e+04, atom= 2031
Step=  570, Dmax= 8.9e-03 nm, Epot= -6.02568e+05 Fma

Step=  685, Dmax= 8.5e-03 nm, Epot= -6.07094e+05 Fmax= 2.04651e+03, atom= 2031
Step=  686, Dmax= 1.0e-02 nm, Epot= -6.07114e+05 Fmax= 1.37833e+04, atom= 2031
Step=  687, Dmax= 1.2e-02 nm, Epot= -6.07217e+05 Fmax= 5.18914e+03, atom= 2031
Step=  689, Dmax= 7.4e-03 nm, Epot= -6.07244e+05 Fmax= 6.17543e+03, atom= 2031
Step=  690, Dmax= 8.8e-03 nm, Epot= -6.07268e+05 Fmax= 7.46333e+03, atom= 2031
Step=  691, Dmax= 1.1e-02 nm, Epot= -6.07288e+05 Fmax= 8.89260e+03, atom= 2031
Step=  692, Dmax= 1.3e-02 nm, Epot= -6.07302e+05 Fmax= 1.07413e+04, atom= 2031
Step=  693, Dmax= 1.5e-02 nm, Epot= -6.07310e+05 Fmax= 1.27845e+04, atom= 2031
Step=  695, Dmax= 9.2e-03 nm, Epot= -6.07408e+05 Fmax= 1.35236e+03, atom= 2031
Step=  696, Dmax= 1.1e-02 nm, Epot= -6.07470e+05 Fmax= 1.56885e+04, atom= 2031
Step=  697, Dmax= 1.3e-02 nm, Epot= -6.07600e+05 Fmax= 4.74897e+03, atom= 2031
Step=  699, Dmax= 7.9e-03 nm, Epot= -6.07621e+05 Fmax= 7.46647e+03, atom= 2031
Step=  700, Dmax= 9.5e-03 nm, Epot= -6.07651e+05 Fma

Step=  816, Dmax= 1.1e-02 nm, Epot= -6.11333e+05 Fmax= 1.01303e+04, atom= 2031
Step=  817, Dmax= 1.3e-02 nm, Epot= -6.11358e+05 Fmax= 1.00463e+04, atom= 2031
Step=  819, Dmax= 7.8e-03 nm, Epot= -6.11422e+05 Fmax= 2.07189e+03, atom= 2031
Step=  820, Dmax= 9.4e-03 nm, Epot= -6.11430e+05 Fmax= 1.24983e+04, atom= 2031
Step=  821, Dmax= 1.1e-02 nm, Epot= -6.11512e+05 Fmax= 4.97718e+03, atom= 2031
Step=  823, Dmax= 6.8e-03 nm, Epot= -6.11535e+05 Fmax= 5.49195e+03, atom= 2031
Step=  824, Dmax= 8.1e-03 nm, Epot= -6.11553e+05 Fmax= 7.07513e+03, atom= 2031
Step=  825, Dmax= 9.8e-03 nm, Epot= -6.11571e+05 Fmax= 7.99449e+03, atom= 2031
Step=  826, Dmax= 1.2e-02 nm, Epot= -6.11579e+05 Fmax= 1.00982e+04, atom= 2031
Step=  827, Dmax= 1.4e-02 nm, Epot= -6.11589e+05 Fmax= 1.15816e+04, atom= 2031
Step=  829, Dmax= 8.4e-03 nm, Epot= -6.11669e+05 Fmax= 1.44471e+03, atom= 2031
Step=  830, Dmax= 1.0e-02 nm, Epot= -6.11698e+05 Fmax= 1.42350e+04, atom= 2031
Step=  831, Dmax= 1.2e-02 nm, Epot= -6.11804e+05 Fma

Step=  948, Dmax= 7.0e-03 nm, Epot= -6.14589e+05 Fmax= 6.53111e+03, atom= 2031
Step=  949, Dmax= 8.4e-03 nm, Epot= -6.14611e+05 Fmax= 6.38875e+03, atom= 2031
Step=  950, Dmax= 1.0e-02 nm, Epot= -6.14614e+05 Fmax= 9.12431e+03, atom= 2031
Step=  951, Dmax= 1.2e-02 nm, Epot= -6.14631e+05 Fmax= 9.46631e+03, atom= 2031
Step=  953, Dmax= 7.2e-03 nm, Epot= -6.14686e+05 Fmax= 1.69931e+03, atom= 2031
Step=  954, Dmax= 8.7e-03 nm, Epot= -6.14698e+05 Fmax= 1.17247e+04, atom= 2031
Step=  955, Dmax= 1.0e-02 nm, Epot= -6.14770e+05 Fmax= 4.37763e+03, atom= 2031
Step=  957, Dmax= 6.2e-03 nm, Epot= -6.14789e+05 Fmax= 5.26828e+03, atom= 2031
Step=  958, Dmax= 7.5e-03 nm, Epot= -6.14805e+05 Fmax= 6.31180e+03, atom= 2031
Step=  959, Dmax= 9.0e-03 nm, Epot= -6.14818e+05 Fmax= 7.57351e+03, atom= 2031
Step=  960, Dmax= 1.1e-02 nm, Epot= -6.14827e+05 Fmax= 9.09788e+03, atom= 2031
Step=  961, Dmax= 1.3e-02 nm, Epot= -6.14831e+05 Fmax= 1.08809e+04, atom= 2031
Step=  963, Dmax= 7.8e-03 nm, Epot= -6.14901e+05 Fma

Step= 1078, Dmax= 7.4e-03 nm, Epot= -6.17428e+05 Fmax= 1.01680e+03, atom= 2031
Step= 1079, Dmax= 8.9e-03 nm, Epot= -6.17464e+05 Fmax= 1.27346e+04, atom= 2031
Step= 1080, Dmax= 1.1e-02 nm, Epot= -6.17549e+05 Fmax= 3.81002e+03, atom= 2031
Step= 1082, Dmax= 6.4e-03 nm, Epot= -6.17559e+05 Fmax= 6.12482e+03, atom= 2031
Step= 1083, Dmax= 7.7e-03 nm, Epot= -6.17579e+05 Fmax= 5.77914e+03, atom= 2031
Step= 1084, Dmax= 9.3e-03 nm, Epot= -6.17580e+05 Fmax= 8.51288e+03, atom= 2031
Step= 1085, Dmax= 1.1e-02 nm, Epot= -6.17596e+05 Fmax= 8.61732e+03, atom= 2031
Step= 1087, Dmax= 6.7e-03 nm, Epot= -6.17643e+05 Fmax= 1.66960e+03, atom= 2031
Step= 1088, Dmax= 8.0e-03 nm, Epot= -6.17649e+05 Fmax= 1.06940e+04, atom= 2031
Step= 1089, Dmax= 9.6e-03 nm, Epot= -6.17710e+05 Fmax= 4.13670e+03, atom= 2031
Step= 1091, Dmax= 5.8e-03 nm, Epot= -6.17725e+05 Fmax= 4.75017e+03, atom= 2031
Step= 1092, Dmax= 6.9e-03 nm, Epot= -6.17738e+05 Fmax= 5.91733e+03, atom= 2031
Step= 1093, Dmax= 8.3e-03 nm, Epot= -6.17750e+05 Fma

_(This'll take a minute or two)_

Run the simulation...

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [15]:
# Run a short simulation
! gmx grompp -f sim.mdp -c em.gro -p system.top -o md.tpr -maxwarn 1
! gmx mdrun -v -deffnm md

                 :-) GROMACS - gmx grompp, 2021.1-MODIFIED (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teem

_(This'll take a minute or two)_

...And check out the result!

In [16]:
# Load the trajectory
mdt_traj = mdt.load("md.xtc", top="system.gro")
# Fix any PBC artifacts
mdt_traj.image_molecules(inplace=True)
# View the trajectory
view = nglview.show_mdtraj(mdt_traj)
view

NGLWidget(max_frame=10)